In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


### **Data Reading**

In [0]:
df = spark.read.format("parquet")\
    .load("abfss://bronze@fmcdatabrickse2e.dfs.core.windows.net/products")
df.display()

In [0]:
df = df.drop("_rescued_data")
df.display()


In [0]:
df.createOrReplaceTempView("products")

### **Functions**

In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_cat.bronze.discount_func(p_price DOUBLE)
RETURNS DOUBLE
RETURN p_price * 0.9;

In [0]:
%sql
SELECT product_id, price, databricks_cat.bronze.discount_func(price) as discounted_price 
FROM products

In [0]:
df.withColumn("discounted_price", expr("databricks_cat.bronze.discount_func(price)")).display()

In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_cat.bronze.upper_func(p_brand STRING)
RETURNS STRING
LANGUAGE PYTHON
AS
$$
  return p_brand.upper()
$$

In [0]:
%sql
SELECT product_id, brand, databricks_cat.bronze.upper_func(brand) as brand_upper
FROM products;

In [0]:
df.write.mode("overwrite")\
    .format("delta")\
    .option("path", "abfss://silver@fmcdatabrickse2e.dfs.core.windows.net/products")\
    .save()

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_cat.silver.products_silver
USING DELTA
LOCATION 'abfss://silver@fmcdatabrickse2e.dfs.core.windows.net/products'